In [ ]:
import h5py
import numpy as np
import os

### Preprocessing k-space data

The raw non-Cartesian k-space data was gridded onto a Cartesian grid using a MATLAB-script. In this notebook, the gridded k-space data gets preprocessed, which includes reconstruction to image space, cropping, and transformation back to k-space, in addition to coil combination. The data is structured as [coils, t, H, W, D]. Due to memory limitations, each temporal slice was processed individually, before being concatted into the final 4D volume.

In [ ]:
def load_data(path):
    """
    Load gridded k-space data from a MATLAB .mat file (version 7.3, HDF5 format).

    Parameters:
    - path: Path to the .mat file.

    Returns:
    - data: Loaded k-space data as a NumPy array.
    """
    with h5py.File(path, 'r') as f:
        data = np.array(f['CK'])
    return data

#### Reconstructing to image space

In [ ]:
def reconstruct(kspace_data, preprocess = False):
    """ 
    Reconstruct image space data from k-space using an inverse Fourier transform.

    Parameters:
    - kspace_data: input k-space data
    - preprocess: Boolean value whether to combine the real and imaginary parts into complex k-space data

    Return:
    - image: image space of k-space data
    """
    
    if preprocess:
        kspace_data = np.squeeze(kspace_data)
        kspace_data = kspace_data['real'] + 1j* kspace_data['imag']
        

    image = np.fft.ifftn(kspace_data, axes=(-3,-2,-1))
    image = np.fft.ifftshift(image, axes=(-3,-2,-1))
    
    return image

#### Coil combination

In [ ]:
def coil_combine(image_data):
    """ 
    Combining coils by root of sum squared (RSS). Input shape: (coils, x, y, z)

    Input:
    - image_data: image data

    Return:
    - combined_colis: a Numpy array with combined coils.
    """
    real_rss = np.sqrt(np.sum(np.real(image_data)**2, axis=0))
    imag_rss = np.sqrt(np.sum(np.imag(image_data)**2, axis=0))
    combined_coils = np.stack([real_rss, imag_rss], axis = 0)
    return combined_coils

#### Cropping in image space

In order to resize the matrix size, the k-space data was transformed to image space, where the image was cropped to match the ground truth size.

In [ ]:
def crop_image_space(image_data, target_size=(128,128,128)):
    """ 
    Center-crop image to the specified target size.

    Parameters:
    - image_data: Input image array with shape (..., H, W, D).
    - target_size: Target size after cropping. Default is (128, 128, 128)

    Returns:
    - cropped_image: Cropped image with shape (..., crop_H, crop_W, crop_D).
    """

    H, W, D = image_data.shape[-3:]   # height, width, depth
    print(f"Shape inside crop function: {image_data.shape}")
    crop_H, crop_W, crop_D = target_size
    start_H = (H-crop_H) // 2
    start_W = (W-crop_W) // 2
    start_D = (D-crop_D) // 2

    cropped_image = image_data[..., start_H:start_H+crop_H, start_W:start_W+crop_W, start_D:start_D+crop_D]

    return cropped_image

In [ ]:
def back_to_kspace(cropped_img):
    """ 
    Convert a cropped image back to k-space using a 3D Fourier transform.
    
    Parameters:
    - cropped_img: Image after cropping

    Return:
    - k_space: Corresponding k-space data after FFT.
    """
    k_space = np.fft.fftshift(cropped_img, axes=(-3,-2,-1))
    k_space = np.fft.fftn(k_space, axes=(-3,-2,-1))
    return k_space

#### Processing of k-space

Combining all the above functions in correct order.

In [ ]:
def process_kspace(data):
    """ 
    Perform all necessary processing steps on k-space data, including reconstruction, 
    coil combination, cropping, and transformation back to k-space.

    Parameters: 
    - data: Loaded k-space data

    Return:
    - k_space: Processed k-space data after all steps.
    
    """
    print(f"Shape before processing: {data.shape}") 

    # Reconstruct each coil and combine them
    coil_recon = [reconstruct(data[coil,:,:,:,:,:], preprocess=True) for coil in range(data.shape[0])]
    coil_recon = coil_combine(coil_recon)

    print(f"Shape before cropping: {coil_recon.shape}")

    # Crop the combined coil image to the target size
    cropped_img = crop_image_space(coil_recon, target_size=(128,128,128))
    print(f"Shape after cropping: {cropped_img.shape}")

    # Convert the cropped image back to k-space
    k_space = back_to_kspace(cropped_img)
    print(k_space.dtype) # Ensuring the data remains complex after the Fourier transform.

    return k_space

In [ ]:
def save_processed_kspace(processed_data, output_dir, filename):
    """ 
    Save the processed k-space data to a specified directory.

    Parameters: 
    - processed_data: The processed k-space data from process_kspace()
    - output_dir: Directory where the processed k-space data will be saved.
    - filename: The filename for the saved data.

    Returns:
    - None: This function saves the file and prints a confirmation message.
    """

    output_path = os.path.join(output_dir, f'processed_{filename}')
    np.save(output_path, processed_data)
    print(f'Saved processed data to {output_path}')


#### Processing and saving

Each participant was processed individually. The function below processes the k-space data and saves the individual time slices in the correct folder.

In [ ]:
def process_and_save(data_dir, output_dir):
    """ 
    Process all k-space data files in a directory and save the processed k-space.

    Parameters:
    - data_dir: Path to the directory containing gridded k-space data files.
    - output_dir: Path to the directory where processed k-space data will be saved.

    Returns:
    - None: This function processes and saves each file in the input directory.
    """

    # List and sort all files in the data directory
    file_list = sorted(os.listdir(data_dir))

    for file in file_list:
        full_path = os.path.join(data_dir, file)
        # Load the data
        current_data = load_data(full_path)
        print(f'Processing {file} with shape {current_data.shape}')

        # Process the data
        processed_kspace = process_kspace(current_data)
        print(f'Done processing {file} with shape {processed_kspace.shape}')

        # Save the processed k-space
        save_processed_kspace(processed_kspace, output_dir, file[:-4])

        # Clean up to free memory 
        del processed_kspace

#### Assembling into 4D data

In [ ]:
def concat_processed(data_dir, output_dir, filename):
    """ 
    Assemble processed 3D k-space timepoints into a single 4D k-space volume

    Parameters:
    - data_dir: Path to the directory containing processed k-space data files.
    - output_dir: Path to the directory where assembled k-space data will be saved.
    - filename: The filename for the saved data.
    
    Returns:
    - None
    """

    # List and sort all files in the data directory
    file_list = sorted(os.listdir(data_dir))

    data_list = []

    for file in file_list:
        full_path = os.path.join(data_dir, file)

        processed_data = np.load(full_path)

        print(f'Loaded file {file} with shape {processed_data.shape}')

        data_list.append(processed_data)

    # Concatenate along time axis
    concatenated_data = np.concatenate(data_list, axis=1)

    output_filename = os.path.join(output_dir, filename)
    np.save(output_filename, concatenated_data)

    # Clean up memory
    data_list.clear()
    del data_list
    del concatenated_data


### Preprocessing and saving all files

Each time point is processed independantly before being assembled into a 4D volume.

In [ ]:
for participant_no in range(first_participant, last_participant):
    folder_dir = f'./matlabcode/gridded_data/0{participant_no}'
    output_dir = f'../data/KSPACE/k-space/preprocessing/0{participant_no}'

    process_and_save(folder_dir, output_dir)

    save_dir = f'../data/KSPACE/k-space/preprocessing/final_data'

    concat_processed(output_dir, save_dir, f"CK_0{participant_no}.npy")